In [50]:
import nltk
import re
import random 
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np  
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian')) 
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/laidhimonthegreen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/laidhimonthegreen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Подготовка 10 предложений

Выбираем только те строчки, в которых есть слово "break"

In [51]:
break_lines = [line for line in open('corpus_eng.txt') if "break" in tokenize(line)]

Выбираем из них 10 рандомных строчек

In [52]:
random.sample(break_lines, 10)

['Mr Hartzer said "a longer-term road map of committed infrastructure investment" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend. \n',
 "Al-Yousef said Fastaqim arrested one of its commanders during battle preparations in Aleppo after a rebel offensive launched last week to break the government's siege of the city's rebel neighborhoods.\n",
 'Beauty Boss: How Rose-Marie Swift Built a Natural-Beauty Empire Dianna Mazzone Share Credit: Courtesy Growing up in small-town Canada, Rose-Marie Swift thought she wanted to become a fashion designer. But after a lucky break lead to a steady paycheck making over dancers at a local Vancouver hotel, the novice designer traded her sewing machine for a set of brushes. Fast-forward twenty years, and Swift was an in-demand makeup artist--working with models like Gisele and Miranda Kerr--when her physical health started to decline. After extensive testing, doctors found her 

Вручную выбираем из этой выборки 10 предложений со словом "break". (Это несложно сделать автоматически, пример будет в конце кода, просто на тот момент было проще достать руками).

1. If you are visiting during Spring Break, be prepared to wait in line for busy attractions like Harry Potter and the Forbidden Journey.

2. My fellow Frenchmen have told me, everywhere, they want to break away from a bureaucratic system which saps their energy.

3. A large group of armed Native Americans break into a federal building in Oregon to protest land use rules. 

4. The 31-year-old will break Michael Atherton's record of 54 Tests as captain during the first match against India.

5. The whole thing arrives like a pretty vibed out, slow-motion take on the original, but Premier flips the whole thing around by the 2:50 mark when he plays off the song's fast paced break.

6. We can’t wait to see what the show has in store for her next–as long as it’s a storyline allowing her to FINALLY catch a break because she could seriously use one.

7. “I’m really good at trains but I can’t seem to figure out people,” he says at one point. “Yes, I broke the law but what do we do about the diagnosis that caused me to break the law?”

8. "Unfortunately, this is the latest in a series of incidents that highlight the need for greater investment in our city facilities," Turner said, pointing to a September water line break at the city\'s main administrative building at 611 Walker, among other episodes.

9. The Cardinal made 11 of 13 shots in the second quarter and led 46-15 at the break.

10. Mr Hartzer said "a longer-term road map of committed infrastructure investment" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend.

Определяем их значения с помощью собственных знаний английского языка и определений из ворднета (уже на этом этапе точность, увы, не гарантирована).
Сначала достаём все значения слова break.

In [53]:
word = 'break'
n = 1
for synset in wn.synsets(word):
    print(str(n) + '. ' + word + ' - ' + synset.definition())
    n += 1

1. break - some abrupt occurrence that interrupts an ongoing activity
2. break - an unexpected piece of good luck
3. break - (geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other
4. break - a personal or social separation (as between opposing factions)
5. break - a pause from doing something (as work)
6. break - the act of breaking something
7. break - a time interval during which there is a temporary cessation of something
8. break - breaking of hard tissue such as bone
9. break - the occurrence of breaking
10. break - an abrupt change in the tone or register of the voice (as at puberty or due to emotion)
11. break - the opening shot that scatters the balls in billiards or pool
12. break - (tennis) a score consisting of winning a game when your opponent was serving
13. break - an act of delaying or interrupting the continuity
14. break - a sudden dash
15. break - any frame in which a bowler fails to make a strike or spare
16. brea

Что ж, их немало. 75. Вау. Посмотрим только на существительные.

In [54]:
word = 'break'
n = 1
for synset in wn.synsets(word, pos=wn.NOUN):
    print(str(n) + '. ' + word + ' - ' + synset.definition())
    n += 1

1. break - some abrupt occurrence that interrupts an ongoing activity
2. break - an unexpected piece of good luck
3. break - (geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other
4. break - a personal or social separation (as between opposing factions)
5. break - a pause from doing something (as work)
6. break - the act of breaking something
7. break - a time interval during which there is a temporary cessation of something
8. break - breaking of hard tissue such as bone
9. break - the occurrence of breaking
10. break - an abrupt change in the tone or register of the voice (as at puberty or due to emotion)
11. break - the opening shot that scatters the balls in billiards or pool
12. break - (tennis) a score consisting of winning a game when your opponent was serving
13. break - an act of delaying or interrupting the continuity
14. break - a sudden dash
15. break - any frame in which a bowler fails to make a strike or spare
16. brea

Уже немного лучше. 

У нас есть следующие предложения, где break -- существительное.

1. If you are visiting during Spring Break, be prepared to wait in line for busy attractions like Harry Potter and the Forbidden Journey.

5. The whole thing arrives like a pretty vibed out, slow-motion take on the original, but Premier flips the whole thing around by the 2:50 mark when he plays off the song's fast paced break.

6. We can’t wait to see what the show has in store for her next–as long as it’s a storyline allowing her to FINALLY catch a break because she could seriously use one.

8. "Unfortunately, this is the latest in a series of incidents that highlight the need for greater investment in our city facilities," Turner said, pointing to a September water line break at the city\'s main administrative building at 611 Walker, among other episodes.

9. The Cardinal made 11 of 13 shots in the second quarter and led 46-15 at the break.

Видно, что значение "перерыв" очень распространенное -- только в этой выборке этому значению соответствуют варианты 1, 6, 9. Впрочем, можно заметить, что значения для существительных 5 и 7 достаточно схожи (перерыв в работе и перерыв по расписанию). Будем считать, что 5 относится к вынужденному перерыву для отдыха, а 7 -- к запланированному перерыву, тогда получаем:

Предложения 1, 9 -- a time interval during which there is a temporary cessation of something. 
Предложение 5 -- some abrupt occurrence that interrupts an ongoing activity (неточно, но лучше из этих вариантов ничего не нашлось). 
Предложение 6 -- good luck, happy chance (an unexpected piece of good luck).
Предложение 8 -- the occurrence of breaking. 

Сделаем то же для глаголов. Чтобы не сбивать нумерацию, пусть значения для глаголов начинаются с 17. 

In [55]:
word = 'break'
n = 17
for synset in wn.synsets(word, pos=wn.VERB):
    print(str(n) + '. ' + word + ' - ' + synset.definition())
    n += 1

17. break - terminate
18. break - become separated into pieces or fragments
19. break - render inoperable or ineffective
20. break - ruin completely
21. break - destroy the integrity of; usually by force; cause to separate into pieces or fragments
22. break - act in disregard of laws, rules, contracts, or promises
23. break - move away or escape suddenly
24. break - scatter or part
25. break - force out or release suddenly and often violently something pent up
26. break - prevent completion
27. break - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act
28. break - make submissive, obedient, or useful
29. break - fail to agree with; be in violation of; as of rules or patterns
30. break - surpass in excellence
31. break - make known to the public information that was previously known only to a few people or that was meant to be kept a secret
32. break - come into being
33. break - stop operating or functioning
34

59 значений (хорошо, хотя бы наречий и прилагательных среди значений слова break нет).  

1. My fellow Frenchmen have told me, everywhere, they want a to break away from a bureaucratic system which saps their energy.

Будем считать, что это 23. break - move away or escape suddenly

2. A large group of armed Native Americans break into a federal building in Oregon to protest land use rules.

Будем считать, что это 27. break - enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act

3. The 31-year-old will break Michael Atherton's record of 54 Tests as captain during the first match against India.

Будем считать, что это 30. break - surpass in excellence

4. “I’m really good at trains but I can’t seem to figure out people,” he says at one point. “Yes, I broke the law but what do we do about the diagnosis that caused me to break the law?”

Будем считать, что это 22. break - act in disregard of laws, rules, contracts, or promises

5. Mr Hartzer said "a longer-term road map of committed infrastructure investment" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend.

Будем считать, что это 20. break - ruin completely

## Алгоритм Леска
Мы как будто закончили с соотнесением наших предложений и значений глагола break. Начнем использовать алгоритм Леска.  

In [56]:
def lesk(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word)):
        definition = tokenize(synset.definition())
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i 
    
    #т.к. нумерация в списке значений была с единицы
    return bestsense + 1

In [57]:
sentences = ["If you are visiting during Spring Break, be prepared to wait in line for busy attractions like Harry Potter and the Forbidden Journey.", 
             "My fellow Frenchmen have told me, everywhere, they want to break away from a bureaucratic system which saps their energy.", 
             "A large group of armed Native Americans break into a federal building in Oregon to protest land use rules.", 
             "The 31-year-old will break Michael Atherton's record of 54 Tests as captain during the first match against India.", 
             "The whole thing arrives like a pretty vibed out, slow-motion take on the original, but Premier flips the whole thing around by the 2:50 mark when he plays off the song's fast paced break.", 
             "We can’t wait to see what the show has in store for her next–as long as it’s a storyline allowing her to FINALLY catch a break because she could seriously use one.", 
             "“I’m really good at trains but I can’t seem to figure out people,” he says at one point. “Yes, I broke the law but what do we do about the diagnosis that caused me to break the law?”", 
             "\"Unfortunately, this is the latest in a series of incidents that highlight the need for greater investment in our city facilities,\" Turner said, pointing to a September water line break at the city's main administrative building at 611 Walker, among other episodes.", 
             "The Cardinal made 11 of 13 shots in the second quarter and led 46-15 at the break.", 
             "Mr Hartzer said \"a longer-term road map of committed infrastructure investment\" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend."]

meanings = [7, 23, 27, 30, 1, 2, 22, 9, 7, 20]
noun_meanings = [7, 1, 2, 9, 7]
verb_meanings = [23, 27, 30, 22, 20]

In [58]:
for i, sentence in enumerate(sentences): 
    print ("Значение по Леску: ", lesk ("break", tokenize(sentence)), "Значение по стандарту: ", meanings[i])

Значение по Леску:  3 Значение по стандарту:  7
Значение по Леску:  3 Значение по стандарту:  23
Значение по Леску:  3 Значение по стандарту:  27
Значение по Леску:  10 Значение по стандарту:  30
Значение по Леску:  35 Значение по стандарту:  1
Значение по Леску:  3 Значение по стандарту:  2
Значение по Леску:  31 Значение по стандарту:  22
Значение по Леску:  3 Значение по стандарту:  9
Значение по Леску:  10 Значение по стандарту:  7
Значение по Леску:  3 Значение по стандарту:  20


Вообще ничего не совпадает. Не очень здорово, но попробуем что-то улучшить. 3 значение кажется очень мусорным, но пока начнем с того, что хотя бы используем частеречную разметку.

## Сортировка по частям речи

In [59]:
#разобьем предложения на те, где break -- глагол и на те, где break -- существительное

noun_sentences = []
verb_sentences = []
for sentence in sentences: 
    for word in (nltk.pos_tag(sentence.split())):
        if word[0].strip(",. ") == "break" or word[0].strip(",. ") == "Break":
            if "NN" in word[1]:
                noun_sentences.append(sentence)
            else:
                verb_sentences.append(sentence)

noun_sentences                

['If you are visiting during Spring Break, be prepared to wait in line for busy attractions like Harry Potter and the Forbidden Journey.',
 "The whole thing arrives like a pretty vibed out, slow-motion take on the original, but Premier flips the whole thing around by the 2:50 mark when he plays off the song's fast paced break.",
 'We can’t wait to see what the show has in store for her next–as long as it’s a storyline allowing her to FINALLY catch a break because she could seriously use one.',
 '"Unfortunately, this is the latest in a series of incidents that highlight the need for greater investment in our city facilities," Turner said, pointing to a September water line break at the city\'s main administrative building at 611 Walker, among other episodes.',
 'The Cardinal made 11 of 13 shots in the second quarter and led 46-15 at the break.']

In [60]:
verb_sentences

['My fellow Frenchmen have told me, everywhere, they want to break away from a bureaucratic system which saps their energy.',
 'A large group of armed Native Americans break into a federal building in Oregon to protest land use rules.',
 "The 31-year-old will break Michael Atherton's record of 54 Tests as captain during the first match against India.",
 '“I’m really good at trains but I can’t seem to figure out people,” he says at one point. “Yes, I broke the law but what do we do about the diagnosis that caused me to break the law?”',
 'Mr Hartzer said "a longer-term road map of committed infrastructure investment" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend.']

Будем анализировать эти предложения раздельно. Предварительно добавим нормализацию.

In [61]:
norm_verb_sentences = [normalize(sentence) for sentence in verb_sentences]
norm_noun_sentences = [normalize(sentence) for sentence in noun_sentences]

**Алгоритм Леска с нормализацией**

Также мы определили, что длина определения слишком мала, а примеры есть не во всех случаях, поэтому мы пробуем использовать слова и из определения, и из примеров.

In [72]:
#вариант леска для глаголов с нормализацией
def lesk_verb(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.VERB)):
        definition = normalize(synset.definition())
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
    
    #значения глаголов идут после существительных
    return bestsense + 17, maxoverlap, maxwordsoverlap

#вариант леска для глаголов с нормализацией и примерами
def lesk_verb_example(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.VERB)): 
        definition = normalize(" ".join(synset.examples()))+normalize(synset.definition()) 
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
            
    
    #значения глаголов идут после существительных
    return bestsense + 17, maxoverlap, maxwordsoverlap

**Уберем стоп-слова**

In [73]:
stopWords = set(stopwords.words('english'))

for i, sentence in enumerate(norm_verb_sentences): 
    sentence = [word for word in sentence if word not in stopWords]
    print ("Значение по Леску, пересечения: ", lesk_verb ("break", sentence), "Значение по стандарту: ", verb_meanings[i])

Значение по Леску, пересечения:  (23, 1, {'away'}) Значение по стандарту:  23
Значение по Леску, пересечения:  (22, 1, {'rules'}) Значение по стандарту:  27
Значение по Леску, пересечения:  (52, 1, {'break'}) Значение по стандарту:  30
Значение по Леску, пересечения:  (31, 1, {'people'}) Значение по стандарту:  22
Значение по Леску, пересечения:  (52, 1, {'break'}) Значение по стандарту:  20


Без примеров уже что-то определяет (правда, слов-пересечений очень мало). Попробуем вариант с примерами.

In [74]:
stopWords = set(stopwords.words('english'))

for i, sentence in enumerate(norm_verb_sentences): 
    sentence = [word for word in sentence if word not in stopWords]
    print ("Значение по Леску, пересечения: ", lesk_verb_example ("break", sentence), "Значение по стандарту: ", verb_meanings[i])

Значение по Леску, пересечения:  (23, 2, {'break', 'away'}) Значение по стандарту:  23
Значение по Леску, пересечения:  (22, 2, {'break', 'rules'}) Значение по стандарту:  27
Значение по Леску, пересечения:  (30, 2, {'record', 'break'}) Значение по стандарту:  30
Значение по Леску, пересечения:  (22, 2, {'break', 'law'}) Значение по стандарту:  22
Значение по Леску, пересечения:  (17, 1, {'break'}) Значение по стандарту:  20


Неплохо, три из пяти. Стоит отметить, что количество пересечений минимальное -- для последнего слова пересечений, кроме самого слова break, нет вовсе. 

Объяснение, почему алгоритм Леска плохо работает для предложений 2 и 5 для глаголов:
    
2. A large group of armed Native Americans break into a federal building in Oregon to protest land use rules.

Верное значение: S: (v) break in, break (enter someone's (virtual or real) property in an unauthorized manner, usually with the intent to steal or commit a violent act) "Someone broke in while I was on vacation"; "They broke into my car and stole my radio!"; "who broke into my account last night?"

Неверное значение: S: (v) transgress, offend, infract, violate, go against, breach, break (act in disregard of laws, rules, contracts, or promises) "offend all laws of humanity"; "violate the basic laws or human civilization"; "break a law"; "break a promise"

Причина: оба значения достаточно близки (оба связаны так или иначе с нарушением каких-либо правил). В предложении присутствует ключевое для неверного значения слово "rules", которое, хотя и не связано в предложении с глаголом "break", все же определяет тематику.

5. Mr Hartzer said "a longer-term road map of committed infrastructure investment" could break the deadlock between consumers and businesses, who are currently looking to each other for cues on whether to spend.

Верное значение: S: (v) break, bust (ruin completely) "He busted my radio!"
Неверное значение: S: (v) interrupt, break (terminate) "She interrupted her pregnancy"; "break a lucky streak"; "break the cycle of poverty"

Забавно: когда я теперь смотрю на неверное значение, мне кажется, что неверное значение на самом деле более верно, чем "полностью разрушить". Думаю, что этот случай можно считать верным определением значения (поставим ему условную 0.5 из-за того, что при создании стандарта я так ошибалась). Тем не менее, проблемы есть: это значение присвоено просто потому, что оно самое распространенное и первое в списке, а слов-пересечений не было найдено ни одного. 

Вряд ли данный результат получится улучшить. Взглянем на существительные.

In [75]:
#вариант леска для существительных с нормализацией и примерами
def lesk_noun_example(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.NOUN)): 
        definition = normalize(" ".join(synset.examples()))+normalize(synset.definition()) 
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
     
    return bestsense + 1, maxoverlap, maxwordsoverlap

In [78]:
for i, sentence in enumerate(norm_noun_sentences): 
    sntnc = [word for word in sentence if word not in stopWords]
    print ("Значение по Леску, пересечения: ", lesk_noun_example ("break", sntnc), "Значение по стандарту: ", noun_meanings[i])

Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  7
Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  1
Значение по Леску, пересечения:  (2, 2, {'finally', 'break'}) Значение по стандарту:  2
Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  9
Значение по Леску, пересечения:  (14, 2, {'break', 'made'}) Значение по стандарту:  7


Два попадания -- одно верное благодаря слову "finally", второе присвоенное автоматически из-за того, что значение было первым в списке. 

In [79]:
for i, sentence in enumerate(norm_noun_sentences):  
    print ("Значение по Леску, пересечения: ", lesk_noun_example ("break", sentence), "Значение по стандарту: ", noun_meanings[i])

Значение по Леску, пересечения:  (10, 4, {'the', 'to', 'break', 'in'}) Значение по стандарту:  7
Значение по Леску, пересечения:  (1, 4, {'when', 'the', 'a', 'break'}) Значение по стандарту:  1
Значение по Леску, пересечения:  (10, 7, {'to', 'the', 'as', 'her', 'break', 'a', 'in'}) Значение по стандарту:  2
Значение по Леску, пересечения:  (10, 7, {'to', 'the', 'of', 'at', 'break', 'a', 'in'}) Значение по стандарту:  9
Значение по Леску, пересечения:  (10, 5, {'the', 'of', 'at', 'break', 'in'}) Значение по стандарту:  7


Без стоп-слов пересечения стали более объемными, но, как можно было предположить, все они на "мусорных" словах.

## Существительные и чудесные костыли для кода

Предположим, что мы не хотим специально искать такие слова среди существительных, которые будут хорошо анализироваться леском. Попробуем расширить поле поиска леска за счет синонимов.

In [80]:
#ищем все синонимы одного слова
def synonyms(word):
    syns = []
    for syn in wn.synsets(word): 
        for l in syn.lemmas(): 
            syns.append(l.name())
            
    return set(syns)

In [81]:
synonyms("break") 

{'bankrupt',
 'better',
 'breach',
 'break',
 'break-dance',
 'break_away',
 'break_dance',
 'break_down',
 'break_in',
 'break_of_serve',
 'break_off',
 'break_out',
 'break_up',
 'breakage',
 'breaking',
 'breakout',
 'bring_out',
 'bump',
 'burst',
 'bust',
 'cave_in',
 'check',
 'collapse',
 'come_apart',
 'conk_out',
 'crack',
 'damp',
 'dampen',
 'demote',
 'develop',
 'die',
 'disclose',
 'discontinue',
 'discover',
 'disruption',
 'divulge',
 'erupt',
 'expose',
 'fail',
 'fall_apart',
 'fall_in',
 'falling_out',
 'fault',
 'faulting',
 'founder',
 'fracture',
 'gaolbreak',
 'gap',
 'geological_fault',
 'get_around',
 'get_out',
 'give',
 'give_away',
 'give_out',
 'give_way',
 'go',
 'go_against',
 'go_bad',
 'good_luck',
 'happy_chance',
 'infract',
 'intermission',
 'intermit',
 'interrupt',
 'interruption',
 'jailbreak',
 'kick_downstairs',
 'let_on',
 'let_out',
 'offend',
 'open_frame',
 'part',
 'pause',
 'prison-breaking',
 'prisonbreak',
 'recess',
 'recrudesce',
 'rel

In [83]:
#сравниваем очень много слов с еще большим количеством слов

def lesk_noun_syns(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.NOUN)): 
        definition = normalize(" ".join(synset.examples()))+normalize(synset.definition()) 
        definition = [word for word in definition if word not in stopWords]
        defslist = [list(synonyms(word)) for word in definition]
        definition = []
        for sent in defslist:
            for word in sent:
                definition.append(word) 
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
            bestdefinition = definition
    
    #значения глаголов идут после существительных
    return bestsense + 1, maxoverlap 

In [84]:
def snttoset(sntns): 
    snt = [word for word in sntns if word not in stopWords]
    sntlist = [list(synonyms(word)) for word in snt]
    snt = []
    for sent in sntlist:
         for word in sent:
            snt.append(word)     
    return set(snt) 

In [85]:
for i, nounsent in enumerate(norm_noun_sentences):
    print ("Значение по Леску, пересечения: ", lesk_noun_syns("break", snttoset(nounsent)), "Значения по стандарту: ", noun_meanings[i])

Значение по Леску, пересечения:  (15, 109) Значения по стандарту:  7
Значение по Леску, пересечения:  (5, 173) Значения по стандарту:  1
Значение по Леску, пересечения:  (2, 116) Значения по стандарту:  2
Значение по Леску, пересечения:  (5, 114) Значения по стандарту:  9
Значение по Леску, пересечения:  (15, 168) Значения по стандарту:  7


Совпало только одно (хотя бы одно!), но пересечений очень много (возможно, среди них очень много мусорных).
Попробуем посмотреть, как там дела у глаголов при таких условиях. 

In [87]:
def lesk_verb_syns(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.VERB)): 
        definition = normalize(" ".join(synset.examples()))+normalize(synset.definition()) 
        definition = [word for word in definition if word not in stopWords]
        defslist = [list(synonyms(word)) for word in definition]
        definition = []
        for sent in defslist:
            for word in sent:
                definition.append(word) 
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
            bestdefinition = definition
    
    #значения глаголов идут после существительных
    return bestsense + 17, maxoverlap 

In [88]:
for i, verbsent in enumerate(norm_verb_sentences):
    print ("Значение по Леску, пересечения: ", lesk_verb_syns("break", snttoset(verbsent)), "Значения по стандарту: ", verb_meanings[i])

Значение по Леску, пересечения:  (23, 107) Значения по стандарту:  23
Значение по Леску, пересечения:  (22, 120) Значения по стандарту:  27
Значение по Леску, пересечения:  (30, 121) Значения по стандарту:  30
Значение по Леску, пересечения:  (45, 118) Значения по стандарту:  22
Значение по Леску, пересечения:  (52, 121) Значения по стандарту:  20


Не так уж много и сломалось, кстати, как минимум первое и третье предложение работают, но лучше не рисковать и с глаголами использовать старый алгоритм. 

Вернемся к существительным и попробуем зафиксировать для каждого из набора значений одинаковую длину (допустим, 100), чтобы на результат не влияла разница между количеством слов в значениях. 

In [89]:
def lesk_noun_syns100(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    contexts = []
    for i, synset in enumerate(wn.synsets(word, pos=wn.NOUN)): 
        definition = normalize(" ".join(synset.examples())+synset.definition())
        definition = [word for word in definition if word not in stopWords]
        defslist = [list(synonyms(word)) for word in definition]
        definition = []
        for sent in defslist:
            for word in sent:
                definition.append(word) 
        definition = set(definition[0:100])
        sentence = set(sentence)
        overlap = len(definition & sentence) 
        words_overlap = definition & sentence
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i  
            maxwordsoverlap = words_overlap
            bestdefinition = definition
    
    #значения глаголов идут после существительных
    return bestsense + 1, maxoverlap 

for i, nounsent in enumerate(norm_noun_sentences):
    print ("Значение по Леску, пересечения: ", lesk_noun_syns100("break", snttoset(nounsent)), "Значения по стандарту: ", noun_meanings[i])

Значение по Леску, пересечения:  (9, 100) Значения по стандарту:  7
Значение по Леску, пересечения:  (5, 100) Значения по стандарту:  1
Значение по Леску, пересечения:  (9, 100) Значения по стандарту:  2
Значение по Леску, пересечения:  (9, 100) Значения по стандарту:  9
Значение по Леску, пересечения:  (9, 100) Значения по стандарту:  7


Забавно: кажется, что количество синонимов для разных значений синсета настолько обширно, что пересечения включают в себя вообще все множество синонимов к словам в определении. Похоже, вариант с синонимами всё-таки неверный и неэффективный, хотя он правильно сработал на двух разных предложениях в двух разных случаях, а точность в 20% -- уже неплохо :) 

## Неожиданно лемматизация

Мне почему-то казалось, что она входила в нормализацию в каком-то виде, но, кажется, всё-таки нет. 

In [90]:
from nltk.stem import WordNetLemmatizer
wnlem = WordNetLemmatizer() 

lemsentences = []
for s in norm_noun_sentences:
    sentence = []
    for word in s: 
        sentence.append((wnlem.lemmatize(word, pos="v")))
    lemsentences.append(sentence)

In [91]:
lemsentences

[['if',
  'you',
  'be',
  'visit',
  'during',
  'spring',
  'break',
  'be',
  'prepare',
  'to',
  'wait',
  'in',
  'line',
  'for',
  'busy',
  'attractions',
  'like',
  'harry',
  'potter',
  'and',
  'the',
  'forbid',
  'journey'],
 ['the',
  'whole',
  'thing',
  'arrive',
  'like',
  'a',
  'pretty',
  'vibed',
  'out',
  'slow-motion',
  'take',
  'on',
  'the',
  'original',
  'but',
  'premier',
  'flip',
  'the',
  'whole',
  'thing',
  'around',
  'by',
  'the',
  '2:50',
  'mark',
  'when',
  'he',
  'play',
  'off',
  'the',
  "song's",
  'fast',
  'pace',
  'break'],
 ['we',
  'can’t',
  'wait',
  'to',
  'see',
  'what',
  'the',
  'show',
  'have',
  'in',
  'store',
  'for',
  'her',
  'next–as',
  'long',
  'as',
  'it’s',
  'a',
  'storyline',
  'allow',
  'her',
  'to',
  'finally',
  'catch',
  'a',
  'break',
  'because',
  'she',
  'could',
  'seriously',
  'use',
  'one'],
 ['unfortunately',
  'this',
  'be',
  'the',
  'latest',
  'in',
  'a',
  'series',


In [92]:
for i, sentence in enumerate(lemsentences): 
    sntnc = [word for word in sentence if word not in stopWords]
    print ("Значение по Леску, пересечения: ", lesk_noun_example ("break", sntnc), "Значение по стандарту: ", noun_meanings[i])

Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  7
Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  1
Значение по Леску, пересечения:  (2, 2, {'finally', 'break'}) Значение по стандарту:  2
Значение по Леску, пересечения:  (1, 1, {'break'}) Значение по стандарту:  9
Значение по Леску, пересечения:  (15, 2, {'make', 'break'}) Значение по стандарту:  7


Ситуация никак не изменилась. Опишем причины неудачи для каждого из предложений.


1. If you are visiting during Spring Break, be prepared to wait in line for busy attractions like Harry Potter and the Forbidden Journey.'

S: (n) pause, intermission, break, interruption, suspension (a time interval during which there is a temporary cessation of something)
Нет примеров, все слова очень абстрактны и не слишком часто встречаются, когда мы говорим о конкретных вещах.
Возможно, отдельное значение "каникулы" и примеры к нему помогли бы. 


2. "The whole thing arrives like a pretty vibed out, slow-motion take on the original, but Premier flips the whole thing around by the 2:50 mark when he plays off the song's fast paced break."

S: (n) interruption, break (some abrupt occurrence that interrupts an ongoing activity) "the telephone is an annoying interruption"; "there was a break in the action when a player was hurt"
Значение достаточно абстрактно, поэтому примеры могут быть из разных областей и не связаны между собой, как в данном случае. 

3. We can’t wait to see what the show has in store for her next–as long as it’s a storyline allowing her to FINALLY catch a break because she could seriously use one.'

Было успешно распознано. 

4. "Unfortunately, this is the latest in a series of incidents that highlight the need for greater investment in our city facilities," Turner said, pointing to a September water line break at the city\'s main administrative building at 611 Walker, among other episodes.'

S: (n) break (the occurrence of breaking) "the break in the dam threatened the valley"
Достаточно короткое определение и пример. Видно, что существуют связанные слова (dam и water, например), но в текущем виде алгоритмы с ними не работают. 

5. The Cardinal made 11 of 13 shots in the second quarter and led 46-15 at the break.

S: (n) pause, intermission, break, interruption, suspension (a time interval during which there is a temporary cessation of something)
Аналогично предложению 1. Также странно, что нет отдельного значения для спортивного перерыва (тогда, возможно, слова "set", "quarter", "shot" могли бы помочь, т.к. они могли бы встречаться в примерах). 

## Выводы и подведение итогов

0. Леск -- не очень. 
1. То, что примеры и определения разной длины, мешает эффективности алгоритма. 
2. Если бы у ворднета было бы для каждого слова по 5-10 частых словосочетаний с ним, это бы, наверное, помогло.

Что помогает (не точно, мерилось только на этих 10 предложениях):
1. Разделение на глаголы и существительные -- уменьшает количество вариантов для каждого слова.
2. Удаление стоп-слов -- убирает из выдачи просто длинные предложения, которые хорошо срабатывают на пересечении разных "the, is, it". 
3. Использование примеров -- добавляет хотя бы немного полезных слов к определениям.

Что почти не помогает:
1. Лемматизация -- для глаголов и так все было хорошо, а для существительных лучше не стало. В английском языке формы (даже broke) меняются не так часто, чтобы значимо повлиять на работу алгоритма Леска.

"Оптимальный" алгоритм: использовать нормализованный леск на примерах и определениях для глаголов (3.5 из 5), леск с синонимами на примерах и определениях для существительных (1 из 5) (работает не слишком эффективно).
Лучше, чем 4.5 из 10, результата не было, если не разделять части речи и использовать для них один алгоритм, то лучший результат 3.5/10 (кажется, с глаголами просто повезло). 

## Бонус

Если мы очень хотим найти предложения, для которых будет нормально работать алгоритм Леска, мы можем попробовать специально проанализировать предложения, которые часто пересекаются с какими-либо определениями (возможно, частое пересечение говорит о близости значений).

In [93]:
#все предложения со словом "break"
break_lines = [line for line in open('corpus_eng.txt') if "break" in tokenize(line)]
all_breaks = []
for line in break_lines:
    line = nltk.sent_tokenize(line)
    line = [sentence for sentence in line if "break" in tokenize(sentence)]
    all_breaks += line

In [94]:
len(all_breaks)

448

In [95]:
def lesk_new(word, sentence): 
    bestsense = 0
    maxoverlap = 0
    maxwords = 0
    contexts = []
    meaning = "no meaning found"
    for i, synset in enumerate(wn.synsets(word)):
        definition = normalize(" ".join(synset.examples()))+normalize(synset.definition()) 
        definition = [word for word in definition if word not in stopWords]
        definition = set(definition)
        sentence = set(sentence)
        overlap = len(definition & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i 
            maxwords = definition & sentence
            meaning = synset.definition()
     
    return bestsense + 1, maxwords, meaning

#достаем те предложения, которые больше чем на одно слово(кроме брейк) пересекаются с определениями и примерами
for i, sentence in enumerate(all_breaks): 
    sentenc = [word for word in tokenize(sentence) if word not in stopWords and word != "break"]
    if (lesk_new("break", sentenc)[1]) != 0:
        if len(lesk_new("break", sentenc)[1]) > 1: 
            print (i+1, sentence, "\n" + "Значение по Леску: ", lesk_new("break", sentenc))

13 GENEVA, Nov 28 The World Trade Organization (WTO) ruled on Monday a tax break from Washington state to help Boeing develop its new 777X jetliner was a prohibited subsidy, in a setback for the U.S. planemaker as it eyes victory in a parallel case against Airbus. 
Значение по Леску:  (43, {'state', 'new'}, 'come forth or begin from a state of latency')
24 In an unprecedented break with custom for the British secret service known as MI5, its director-general Andrew Parker was reportedly the first-ever incumbent head of the organization to give a full-court media interview . 
Значение по Леску:  (31, {'known', 'secret'}, 'make known to the public information that was previously known only to a few people or that was meant to be kept a secret')
30 Women take on surf contest for 1st time 1 / 1 Back to Gallery HONOLULU (AP) — Charging into violent waves at a Maui surf break known as "Jaws," a group of women made history Friday as they competed for the first time in the World Surf League 's

268 Hello everyone so me and my ex boyfriend dated for a month I was really happy with him and we got really close then two days after our one month Anniversary he told me that he was seeing another girl in the time we dated and that he told her to stay away from him and then he asked me for distance and that we break... show more Hello everyone so me and my ex boyfriend dated for a month I was really happy with him and we got really close then two days after our one month Anniversary he told me that he was seeing another girl in the time we dated and that he told her to stay away from him and then he asked me for distance and that we break up because he doesn't want to break any girls heart and that he won't choose the one over the other but he really want me its just what he did to me was wrong and I don't deserve what he did to , he asked to be friends with me but ever since we broke up its just akkward if we see each other we will just stare and walk past I really really want him b

405 The 2014 Thanksgiving game saw the Eagles dismantle Dallas 33-10, and last year’s Thanksgiving game saw Romo break his collarbone for the second time in the same season in a 33-14 loss. 
Значение по Леску:  (12, {'second', 'game'}, '(tennis) a score consisting of winning a game when your opponent was serving')
412 Associated Press - 4 hours ago PHILADELPHIA (AP) — The Latest on strike by about 4,700 transit workers in Philadelphia (all times local): … More » Chart: The Aussie is poised for a breakout but it could go either way CNBC - 5 hours ago Its too early to know which direction the Aussie will break but when it does, there's a high probability it will move towards projection targets. 
Значение по Леску:  (23, {'high', 'move'}, 'move away or escape suddenly')
420 ... more Photo: Taylor Jewell, Taylor Jewell/Invision/AP Q&A: Redmayne on finally getting his Harry Potter shot 1  /  5 Back to Gallery NEW YORK (AP) — After spending two straight falls consumed by awards season, Eddie

Есть ли среди них что-то правильно определенное? Местами да, вероятно, 10 предложений набрать можно без нажатия на кнопку рандом и надежду, что повезет.

Например:

24 In an unprecedented break with custom for the British secret service known as MI5, its director-general Andrew Parker was reportedly the first-ever incumbent head of the organization to give a full-court media interview . 
Значение по Леску:  (31, {'secret', 'known'}, 'make known to the public information that was previously known only to a few people or that was meant to be kept a secret')

Или: 

56 He got his big break in 1997 when Radio City and Rockefeller Center got on board. 
Значение по Леску:  (2, {'big', 'got'}, 'an unexpected piece of good luck')

Или:

313 The actress recently took a break from filming Suits , and it seems that she's using the time to get up close and personal with the world's favourite ginger. 
Значение по Леску:  (5, {'took', 'time'}, 'a pause from doing something (as work)')

Тем не менее, даже на больших пересечениях значение чаще определяется неверно: для настолько большого количества часто схожих значений сложно уловить разницу. 